# Data Collection Experimentation
- The announcement of the USAID funding cuts was made on March 28, 2025
## 1. Environments
- tweepy, PRAW requests (pip install)

## 2. Reddit Data collection
- Create connection with Reddit: [link to create script->(https://www.reddit.com/prefs/apps)](https://www.reddit.com/prefs/apps)
- Get `client_id`, `client_secret`
```
reddit = praw.Reddit(
    client_id='tWd763iMgmjp8YamFF96Wg',
    client_secret='	XmaO9oO_kioW-8DzCLipxz-A4hffSA',
    user_agent='usaid-sentiment-KE'
)

```

In [ ]:
import praw
import pandas as pd
from datetime import datetime, timezone

reddit = praw.Reddit(
    client_id='tWd763iMgmjp8YamFF96Wg',
    client_secret='XmaO9oO_kioW-8DzCLipxz-A4hffSA',
    user_agent='usaid-sentiment-KE'
)

# --- PARAMETERS ---
subreddits = ['Kenya', 'EastAfrica']

"""keywords = [
    "USAID", "usaid", 
    "foreign aid", "foreign assistance", 
    "donor", "donour", 
    "funding", "funds", 
    "budget cuts", "aid cuts", 
    "development aid", 
    "healthcare", "health care", 
    "NGOs", "nonprofits", "non-profits"
]"""


keywords = [
    "USAID", "usaid", 
    "foreign aid", "foreign funding"  
]

# Combine keywords and phrases
search_terms = keywords

# Earliest date (after funding cuts) → March 28, 2025
cutoff_date = datetime(2025, 3, 28, tzinfo=timezone.utc).timestamp()

# --- SCRAPING ---
data = []

for sub in subreddits:
    subreddit = reddit.subreddit(sub)
    print(f"Searching r/{sub}...")
    
    for term in search_terms:
        try:
            for post in subreddit.search(term, sort='new', limit=200):
                if post.created_utc < cutoff_date:
                    continue  # skip posts before March 28, 2025
                
                data.append({
                    'subreddit': sub,
                    'search_term': term,
                    'title': post.title,
                    'text': post.selftext,
                    'created_utc': post.created_utc,
                    'created_date': datetime.fromtimestamp(post.created_utc),
                    'score': post.score,
                    'num_comments': post.num_comments,
                    'permalink': f"https://reddit.com{post.permalink}",
                    'url': post.url
                })
        except Exception as e:
            print(f"Error searching term '{term}' in r/{sub}: {e}")

# --- SAVE TO CSV ---
df = pd.DataFrame(data)
df['created_date'] = pd.to_datetime(df['created_utc'], unit='s')
df.to_csv('../data/raw/leo_reddit_posts.csv', index=False)

print(f"✅ Scraped {len(df)} posts. Saved to ../data/raw/leo_reddit_posts.csv'.")


Searching r/Kenya...
Searching r/EastAfrica...
✅ Scraped 24 posts. Saved to ../data/raw/leo_reddit_posts.csv'.


In [2]:
df.sample(random_state=42, n=10)

,subreddit,search_term,title,text,created_utc,created_date,score,num_comments,permalink,url
8,Kenya,usaid,Economy,For the experts in matters economy and finance...,1.743959e+09,2025-04-06 17:01:41,1,15,https://reddit.com/r/Kenya/comments/1jsytyp/ec...,https://www.reddit.com/r/Kenya/comments/1jsyty...
16,Kenya,foreign funding,Be very cautious of the UAE,Kasongo has been cozying up to the UAE recentl...,1.748512e+09,2025-05-29 09:51:51,31,13,https://reddit.com/r/Kenya/comments/1ky6sma/be...,https://www.reddit.com/r/Kenya/comments/1ky6sm...
0,Kenya,USAID,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,1.747235e+09,2025-05-14 15:11:02,11,32,https://reddit.com/r/Kenya/comments/1kmhn87/us...,https://www.reddit.com/r/Kenya/comments/1kmhn8...
18,Kenya,foreign funding,Daily Nation,,1.747811e+09,2025-05-21 07:09:24,1,8,https://reddit.com/r/Kenya/comments/1krrnpb/da...,https://www.reddit.com/gallery/1krrnpb
11,Kenya,foreign aid,Is There a Better Way to Fund Africa’s Infrast...,I'm researching a fintech concept rooted in a ...,1.745161e+09,2025-04-20 14:49:50,10,9,https://reddit.com/r/Kenya/comments/1k3o7to/is...,https://www.reddit.com/r/Kenya/comments/1k3o7t...
9,Kenya,usaid,EX-USAID people!! Let's talk,Are you still in contact with the organisation...,1.743880e+09,2025-04-05 19:09:10,2,0,https://reddit.com/r/Kenya/comments/1jsb149/ex...,https://www.reddit.com/r/Kenya/comments/1jsb14...
13,Kenya,foreign aid,Kibaki ALSO failed us,\nThere is a tendency to over-exaggerate the p...,1.743470e+09,2025-04-01 01:12:42,118,124,https://reddit.com/r/Kenya/comments/1jojl2f/ki...,https://www.reddit.com/r/Kenya/comments/1jojl2...
1,Kenya,USAID,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,1.744723e+09,2025-04-15 13:16:53,3,5,https://reddit.com/r/Kenya/comments/1jzrn2s/us...,https://www.reddit.com/r/Kenya/comments/1jzrn2...
21,Kenya,foreign funding,"Like It or Not, Here's Why Ruto Will Win in 20...","CALL ME WHATEVER YOU WANT, BUT HERE'S THE BARE...",1.745140e+09,2025-04-20 09:06:35,0,9,https://reddit.com/r/Kenya/comments/1k3ijdu/li...,https://www.reddit.com/r/Kenya/comments/1k3ijd...
5,Kenya,usaid,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,1.747235e+09,2025-05-14 15:11:02,13,32,https://reddit.com/r/Kenya/comments/1kmhn87/us...,https://www.reddit.com/r/Kenya/comments/1kmhn8...


In [7]:
sample = df.sample(n=5,random_state= 42)
display(sample)

,subreddit,search_term,title,text,created_utc,created_date,score,num_comments,permalink,url
8,Kenya,usaid,Economy,For the experts in matters economy and finance...,1.743959e+09,2025-04-06 17:01:41,1,15,https://reddit.com/r/Kenya/comments/1jsytyp/ec...,https://www.reddit.com/r/Kenya/comments/1jsyty...
16,Kenya,foreign funding,Be very cautious of the UAE,Kasongo has been cozying up to the UAE recentl...,1.748512e+09,2025-05-29 09:51:51,31,13,https://reddit.com/r/Kenya/comments/1ky6sma/be...,https://www.reddit.com/r/Kenya/comments/1ky6sm...
0,Kenya,USAID,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,1.747235e+09,2025-05-14 15:11:02,11,32,https://reddit.com/r/Kenya/comments/1kmhn87/us...,https://www.reddit.com/r/Kenya/comments/1kmhn8...
18,Kenya,foreign funding,Daily Nation,,1.747811e+09,2025-05-21 07:09:24,1,8,https://reddit.com/r/Kenya/comments/1krrnpb/da...,https://www.reddit.com/gallery/1krrnpb
11,Kenya,foreign aid,Is There a Better Way to Fund Africa’s Infrast...,I'm researching a fintech concept rooted in a ...,1.745161e+09,2025-04-20 14:49:50,10,9,https://reddit.com/r/Kenya/comments/1k3o7to/is...,https://www.reddit.com/r/Kenya/comments/1k3o7t...


In [11]:
sample_text= sample['text'].to_list()
for x in sample_text:
    display(x)

'For the experts in matters economy and finance I ask this politely(mnielezee Kama mtoto tafadhali). How is our country still semi functional? Everyday we hear cases of billions lost here billions lost there. Sometime there was reports of I think 1.3 trillion irregularly withdrawn from the treasury, the dollar has surprisingly been stable at around 129 despite all this and there was the case where funding would be halted by the USAID. How has the economy not crashed yet? Is it normal to lose a third of the budget and still have a running country?'

"Kasongo has been cozying up to the UAE recently and as Kenyans we should be very careful here, if you look at their foreign policy they a pattern of fostering chaos and undermining democracy and legitimate governments.\n\n- In Sudan they fund and support the RSF ,in fact there are reports that they are the ones who pushed the RSF into launching the war.\n- In Somalia they support the breakaway region of Somaliland.\n- In Libya they fund and support the warlord, Khalifa Haftar.\n- In Egypt they orchestrated a coup to overthrow Morsy, the only democratically elected leader in Egypt.\n\nI don't know but who's to say that they will not try and help Kasongo in subverting the 2027 elections? After all they wouldn't wanna lose their logistics hub.  As the Swahili say 'Ukiona cha mwenzako kinanyolewa ,chako tia maji' and btw all those countries you see above all thought it couldn't happen to them."

'My neighbour’s wife was a very big shot in USAID and has now lost her job. Children have been removed from big private school. Husband is a big guy at PWC. Lifestyle changes are occurring rapidly as her income has vanished. Thousands of her USAID coworkers were sent home with no salaries. \n\nUSAID Vendors, contractors, non-profits that received funding from them have all been left in a lurch. Sasa machozi zimeanza. \n\nNext is empty apartments around “high class” areas.\n\nUN is laying people off left right and center.\n\nAdditionally, public assistance programs in the Europe and America are being slashed so remittances by a certain sector are falling.\n\nIf you think things are hard, ngojeni mpaka December. A lot of your highlife hotspots are about to close. A lot of these restaurants are about to close. \n\nCrime shall return so please rudini mashambani mulime.\n\nAvoid Mombasa, Lamu and malls.\n\n\n'

''

'I\'m researching a fintech concept rooted in a simple but powerful idea: What if African citizens could directly micro-invest in their own infrastructure and economic development — from as little as $1 — instead of relying so heavily on foreign loans or aid?\n\nThe idea is inspired by:\n\nEthiopia\'s Renaissance Dam, where despite China funding most of the $5B project, citizens contributed around $1B through bonds and mobile payments. It was a unifying act of nation-building.\n\nDenmark’s wind cooperatives, where tens of thousands of Danes co-own wind turbines, investing small amounts and earning steady returns from green energy sales.\n\nArla Foods, one of the world’s largest dairy companies, is owned by thousands of farmer-members across Europe.\n\nPark Slope Food Co-op (Brooklyn, USA) – over 17,000 members run and own this highly successful grocery store. Members contribute labor and share in decision-making and cost savings — a small-scale but high-functioning democratic economic 

- As seen above, some texts in our dataset contains mixed languages; we shall therefore have some preprocessing before analysis